In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path, start_page):
    text = ""
    pdf_document = fitz.open(pdf_path)
    for page_num in range(start_page - 1, len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text

pdf_path = '/kaggle/input/fa-manual/FA-manual.pdf'
start_page = 14  # Specify the starting page number
text = extract_text_from_pdf(pdf_path, start_page)


In [ ]:
from sentence_transformers import SentenceTransformer
from chromadb import Client
import torch
# Initialize SentenceTransformer and Chroma client
model = SentenceTransformer('all-MiniLM-L6-v2')
client = Client()
collection = client.create_collection(name='first_aid_knowledge_base')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

def generate_and_store_embeddings(text):
    chunks = text.split('\n')  # Example chunking, adjust as necessary
    embeddings = model.encode(chunks)
    
    # Create lists for IDs, embeddings, and metadata
    ids = [str(i) for i in range(len(chunks))]
    metadata = [{'text': chunk} for chunk in chunks]
    
    # Add data to the collection
    collection.add(
        ids=ids,
        embeddings=embeddings.tolist(),  # Convert to list if needed
        metadatas=metadata
    )

generate_and_store_embeddings(text)


In [ ]:
import asyncio
import time
import re
from sentence_transformers import SentenceTransformer
from chromadb import Client
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Initialize SentenceTransformer, Chroma client, and GPT-2 model
model = SentenceTransformer('all-MiniLM-L6-v2')
client = Client()
collection = client.get_collection('first_aid_knowledge_base')

gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

async def retrieve_context_and_generate_response(query):
    
    # Encode the query
    query_embedding = model.encode([query])[0].tolist()
    
    # Query the Chroma collection
    results = collection.query(query_embeddings=[query_embedding], n_results=10)  # Adjust parameters as needed
    
    # Process results
    context_ = results['metadatas'][0]
    context = ''
    
    if context_:
        for item in context_:
            if 'text' in item:
                context += item['text'] + " "

    # Generate a response using GPT-2 with the retrieved context
    input_text = f"Based on the information: {context}\n\nQuestion: {query}\n\nAnswer:"
    
    # Tokenize input text
    encoding = gpt2_tokenizer(input_text, return_tensors='pt', truncation=True, max_length=1024)
    inputs = encoding['input_ids']
    attention_mask = encoding.get('attention_mask', None)
    
    # Generate text
    outputs = gpt2_model.generate(
        inputs, 
        attention_mask=attention_mask, 
        max_length=400, 
        num_return_sequences=1, 
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        num_beams=5,
        pad_token_id=gpt2_tokenizer.eos_token_id 
    )
    
    response = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = ''.join(response.split('\n\n')[-1].split('.')[:-1])
    response = re.sub(r'\d+', '', response)
    return response

def determine_intent(user_input):
    emergency_keywords = ["emergency", "urgent", "help", "immediate", "critical"]
    message_keywords = ["message", "leave a message", "note", "notify", "contact"]

    if any(keyword in user_input.lower() for keyword in emergency_keywords):
        return "emergency"
    elif any(keyword in user_input.lower() for keyword in message_keywords):
        return "leave_message"
    else:
        return "unknown"

def needs_treatment_query(user_input):
    treatment_keywords = ["treatment", "help with", "how to treat", "what to do"]
    return any(keyword in user_input.lower() for keyword in treatment_keywords)

async def chatbot():
    while True:
        user_input = input("How can I assist you? ").strip().lower()
        intent = determine_intent(user_input)
        
        if intent == "emergency":
            await handle_emergency()
            break  # End conversation after handling emergency
        elif intent == "leave_message":
            await leave_message()
            break  # End conversation after leaving a message
        else:
            print("I don’t understand that. Please specify if it is an emergency or if you want to leave a message for the doctor.")
            # Continue the loop for another input

async def handle_emergency():
    emergency_description = input("Please describe the emergency: ").strip().lower()
    
    if not needs_treatment_query(emergency_description):
        emergency_description = "How to treat " + emergency_description
    
    # Start the timer and the background generation task
    start_time = time.time()
    response_task = asyncio.create_task(retrieve_context_and_generate_response(emergency_description))
    
    await get_user_address(response_task, start_time)

async def get_user_address(response_task, start_time):
    user_address = input("Please provide your address: ").strip()
    print("The doctor shall soon be reaching your location.")
    
    await post_address_assistance(response_task, start_time)

async def post_address_assistance(response_task, start_time):
    user_input = input("How may I assist you further? ").strip().lower()
    
    if any(word in user_input for word in ["wait", "okay", "yes", "alright"]):
        print("Thank you for your patience. Dr. Adrin will be with you shortly.")
        return

    
    elapsed_time = time.time() - start_time
    if elapsed_time<15:
        print("Just a moment...")
    else:
        while elapsed_time < 15:
            elapsed_time = time.time() - start_time
            if elapsed_time >= 15:
                break
            await asyncio.sleep(1)
    
    if response_task.done():
        response = await response_task
        print(f"“Don’t worry, please follow these steps, Dr. Adrin will be with you shortly”\n\n{response}")
    else:
        print("Please hold a moment while we generate a response...")
        response = await response_task
        print(f"“Don’t worry, please follow these steps, Dr. Adrin will be with you shortly”\n\n{response}")

async def leave_message():
    user_message = input("Please leave your message for the doctor: ").strip()
    print("Your message has been recorded. The doctor will get back to you soon.")




In [10]:
# Start the chatbot
asyncio.run(chatbot())

How can I assist you?  I need help
Please describe the emergency:  My friend is having a sprain in his ankle
Please provide your address:  IIT Patna


The doctor shall soon be reaching your location.


How may I assist you further?  Please call the doctor at immediate


Please hold a moment while we generate a response...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


“Don’t worry, please follow these steps, Dr. Adrin will be with you shortly”

Answer: If you have a sprain in your ankle, you may be able to help your friend heal by following the following steps:  Ask your doctor if he or she can help you  Tell him or her about your condition  Take a blood test  Call your local emergency room  Talk to a doctor  See a specialist  Have a physical exam  Go to the doctor's office  Get a CT scan  Make a note of the location of your injury  Write down your symptoms  Check your blood pressure  Look for any signs or symptoms of pain or swelling  If there is any swelling or pain, call your physician immediately  Be sure to talk to your family and friends  Keep in touch with your friends and family members  Do not take any medications  Don't drink alcohol  Stay hydrated  Sleep well  Drink plenty of water  Eat a healthy diet  Exercise regularly  Wear a good pair of shoes  Clean up after yourself  Give yourself time to rest  Relax and enjoy the day
